In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split


In [5]:
# データの読み込み
data = pd.read_csv('../../data/preprocessed_data.csv')

# 特徴量とターゲット変数の分割
X = data.drop('dengue', axis=1)  
y = data['dengue']

In [6]:
# データをテンソルに変換
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
# ニューラルネットワークの定義
class BinaryClassificationNN(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassificationNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)  # 1つ目の全結合層
        self.fc2 = nn.Linear(16, 8)          # 2つ目の全結合層
        self.fc3 = nn.Linear(8, 1)           # 出力層（1つのニューロン）
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))          # 1つ目の層でReLU活性化関数
        x = torch.relu(self.fc2(x))          # 2つ目の層でReLU活性化関数
        x = torch.sigmoid(self.fc3(x))       # 最後の層でシグモイド関数
        return x


In [ ]:
# モデルの初期化
input_size = X_train.shape[1]  # 特徴量の数
model = BinaryClassificationNN(input_size)

# 損失関数と最適化手法の設定
criterion = nn.BCELoss()  # バイナリクロスエントロピー損失関数
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# トレーニング
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # 訓練モード
    optimizer.zero_grad()  # 勾配の初期化
    outputs = model(X_train)  # 順伝播
    loss = criterion(outputs, y_train)  # 損失計算
    loss.backward()  # 逆伝播
    optimizer.step()  # パラメータの更新
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
# テストデータでの評価
model.eval()  # 評価モード
with torch.no_grad():
    predictions = model(X_test)
    predictions = predictions.round()  # 出力を0または1に丸める
    accuracy = (predictions == y_test).float().mean()
    print(f'Accuracy: {accuracy:.4f}')
